### Start and Run All

In [2]:
import os
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
# from datetime import date, timedelta
from datetime import datetime
from time import sleep, strftime

# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-21 20:04:49


In [3]:
url = "https://www.set.or.th/en/market/product/stock/quote/"
percent_span = '' 

In [4]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [5]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"Obsidian path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
Obsidian path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [6]:
file_name = "name-ttl.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file : {input_file}") 

Input file : C:\Users\PC1\OneDrive\A5\Data\name-ttl.csv


In [7]:
file_name = "price-hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [8]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A5\Data\price-hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\price-hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\price-hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\price-hilo.csv


In [9]:
def cvt_str_to_flt(input_str):
    # Check for the special pattern and return 0
    if input_str == '(-)':
        return 0
    
    # Remove parentheses and percent sign
    cleaned_str = input_str.replace('(', '').replace(')', '').replace('%', '')

    # Convert to float
    try:
        return float(cleaned_str)
    except ValueError:
        # Handle the case where the conversion fails
        return None
 

In [10]:
def get_stock_data(in_df):

    in_df.columns = ['name']
    
    output_columns = ['name', 'price', 'today_low', 'today_high', 'year_low', 'year_high', 'percent', 'updated_at']
    # Initialize an empty DataFrame with specified columns
    out_df = pd.DataFrame(columns=output_columns)
    
    for index, row in in_df.iterrows():
        name = str(row['name']).upper()
        response = request.urlopen(url + name + '/price')
        html_page = BeautifulSoup(response, 'html.parser')
        
  #      price = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info').text.strip()        
  #      if (price == '-'):
  #          price = '0.00'        

        price_div = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info')
        if price_div:
            price = price_div.text.strip()
        else:
            print(f"Price div not found for {name}")
            price = '0.00'     
            
        # Try to find the <h3> tag with the class 'theme-success'
        h3_tag = html_page.find('h3', class_='theme-success')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-danger')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)    
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-normal')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)   
                
        percent = cvt_str_to_flt(percent_span)
                
        values_low = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder me-auto lh-1')
        today_low = values_low[0].text
        if (today_low == '-'):
            today_low = '0.00'
        
        values_high = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder lh-1')
        today_high = values_high[0].text
        if (today_high == '-'):
            today_high = '0.00'         
        
        year_low = values_low[1].text        
        year_high = values_high[1].text     
        
        # Find the div with the specific class, then find the span within it
        div = html_page.find('div', class_='price-detail-date d-flex align-items-center text-nowrap fs-12px')
        if div:
            span = div.find('span')
            if span:
                text = span.get_text()
#                 print(text)
            else:
                print("Span not found")
        else:
            print("Div not found")
            
        # Extract the date part from the string
        # Assuming the date is always in the format 'dd Mon yyyy'
        date_str = ' '.join(text.split()[2:5])

        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, '%d %b %Y')

        # Format the datetime object to 'yyyy-mm-dd'
        updated_at = date_obj.strftime('%Y-%m-%d')

#         print(updated_at)
        
        # Append row to the DataFrame and print the row
        out_df.loc[len(out_df)] = [name, price, today_low, today_high, year_low, year_high, percent, updated_at]
#       print(out_df.iloc[-1].to_string(header=False, index=False)) 
        print(','.join(map(str, out_df.iloc[-1])))
        sleep(1)
        
    return out_df  # return the final dataframe with all values populated

In [11]:
start_time = strftime("%I:%M %p")
print('Start at: ', start_time)

Start at:  08:04 PM


In [12]:
df = pd.read_csv(input_file, header=None)
out_df = get_stock_data(df)

ACE,1.29,1.27,1.29,1.23,1.52,0.0,2025-02-21
ADVANC,286.00,280.00,289.00,193.00,307.00,1.06,2025-02-21
AEONTS,110.50,108.50,110.50,103.50,168.50,0.45,2025-02-21
AH,14.00,13.90,14.00,13.70,26.00,0.0,2025-02-21
AIE,1.66,1.66,1.76,0.72,1.65,5.06,2025-02-21
AIMIRT,10.70,10.60,10.80,10.20,11.20,0.0,2025-02-21
AIT,4.66,4.60,4.66,3.74,5.05,1.75,2025-02-21
AJ,2.36,2.34,2.42,2.26,7.15,-1.67,2025-02-21
AMATA,25.25,24.90,25.50,20.10,31.00,1.41,2025-02-21
ANAN,0.45,0.44,0.46,0.40,0.94,2.27,2025-02-21
AOT,43.50,43.50,44.25,41.50,67.50,-0.57,2025-02-21
AP,8.45,8.15,8.55,7.50,11.20,3.05,2025-02-21
ASIAN,7.95,7.75,7.95,6.35,11.20,1.27,2025-02-21
ASK,7.20,7.15,7.35,7.00,19.20,-0.69,2025-02-21
ASP,2.30,2.26,2.30,2.12,2.88,0.0,2025-02-21
ASW,7.20,7.10,7.50,7.35,8.45,-4.64,2025-02-21
AWC,3.14,3.12,3.16,2.88,4.58,0.0,2025-02-21
BA,20.90,20.70,21.10,15.30,26.00,0.0,2025-02-21
BAM,6.45,6.20,6.55,5.35,10.40,3.2,2025-02-21
BANPU,4.42,4.40,4.50,4.38,7.50,-1.78,2025-02-21
BAY,23.50,23.50,23.70,22.20,29.00,-0.84,2

In [13]:
end_time = strftime("%I:%M %p")
print('End at: ', end_time)

End at:  08:10 PM


In [14]:
out_df.query('price == "-"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
25,BCT,-,0.00,0.00,53.00,76.75,0.0,2025-02-21
169,SPC,-,0.00,0.00,57.00,63.75,0.0,2025-02-21


In [15]:
out_df.shape

(221, 8)

In [16]:
out_df = out_df[out_df['name'] != 'BCT']

In [17]:
out_df = out_df[out_df['name'] != 'LPF']

In [18]:
out_df = out_df[out_df['name'] != 'TYCN']

In [19]:
out_df = out_df[out_df['name'] != 'GRAMMY']

In [20]:
out_df = out_df[out_df['name'] != 'MST']

In [24]:
out_df = out_df[out_df['name'] != 'SPC']

In [26]:
out_df.shape

(216, 8)

In [28]:
out_df['price'] = out_df['price'].astype('float')
out_df['today_low'] = out_df['today_low'].astype('float')
out_df['today_high'] = out_df['today_high'].astype('float')
out_df['year_low'] = out_df['year_low'].astype('float')
out_df['year_high'] = out_df['year_high'].astype('float')
out_df['percent'] = out_df['percent'].astype('float')

In [30]:
sorted_out_df = out_df.sort_values('percent')

In [32]:
sorted_out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
97,KEX,1.14,1.14,1.21,1.19,6.50,-8.80,2025-02-21
126,OSP,14.20,14.20,14.90,14.40,25.00,-4.70,2025-02-21
140,PTTGC,18.30,18.00,19.30,18.60,41.25,-4.69,2025-02-21
15,ASW,7.20,7.10,7.50,7.35,8.45,-4.64,2025-02-21
90,IRPC,0.98,0.97,1.02,1.00,2.06,-2.97,2025-02-21
53,DCC,1.45,1.45,1.49,1.48,2.16,-2.03,2025-02-21
157,SCGP,14.30,14.20,14.70,14.50,35.25,-1.38,2025-02-21
155,SCC,141.50,140.50,143.50,142.50,275.00,-1.05,2025-02-21
214,VNG,2.78,2.74,2.78,2.76,3.92,0.00,2025-02-21
66,FPT,7.00,6.95,7.05,7.00,15.40,0.00,2025-02-21


In [34]:
sorted_out_df.query('name == "MCS"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
116,MCS,7.25,7.2,7.45,5.8,8.2,6.62,2025-02-21


In [36]:
out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
15,ASW,7.20,7.10,7.50,7.35,8.45,-4.64,2025-02-21
53,DCC,1.45,1.45,1.49,1.48,2.16,-2.03,2025-02-21
66,FPT,7.00,6.95,7.05,7.00,15.40,0.00,2025-02-21
90,IRPC,0.98,0.97,1.02,1.00,2.06,-2.97,2025-02-21
97,KEX,1.14,1.14,1.21,1.19,6.50,-8.80,2025-02-21
100,KSL,1.75,1.25,1.76,1.57,2.60,1.74,2025-02-21
126,OSP,14.20,14.20,14.90,14.40,25.00,-4.70,2025-02-21
140,PTTGC,18.30,18.00,19.30,18.60,41.25,-4.69,2025-02-21
155,SCC,141.50,140.50,143.50,142.50,275.00,-1.05,2025-02-21
157,SCGP,14.30,14.20,14.70,14.50,35.25,-1.38,2025-02-21


In [38]:
sorted_out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
99,KKP,57.00,54.50,57.50,37.00,56.25,2.70,2025-02-21
4,AIE,1.66,1.66,1.76,0.72,1.65,5.06,2025-02-21


In [40]:
out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
4,AIE,1.66,1.66,1.76,0.72,1.65,5.06,2025-02-21
99,KKP,57.00,54.50,57.50,37.00,56.25,2.70,2025-02-21


In [42]:
file_name = "Price-Hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\Price-Hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-Hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-Hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-Hilo.csv


In [46]:
out_df.to_csv(output_file, index=False)  
out_df.to_csv(god_file, index=False) 
out_df.to_csv(icd_file, index=False)  
out_df.to_csv(osd_file, index=False) 

In [48]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-21 20:21:41
